# Requests

Nejdřiv si nainstaluj Requests, knihovnu pro webové klienty:

    $ pip install requests
    
A pak v Pythonu:

In [1]:
import requests

Knihovna Requests ti umožní stahovat webové stránky serverů na Internetu, podobně jako to dělá webový prohlížeč.
(Prohlížeče umí pak stránky umí i zobrazis, Requests je jen stahuje.)

Zkusíme si to napřed se stránkou http://python.cz:

In [2]:
odpoved = requests.get('http://python.cz')

Requests za tebe vytvořil dotaz, který poslal serveru jménem "python.cz". V dotazu jsou "hlavičky", které určují co přesně stáhnout:

In [3]:
print(odpoved.request.headers)

{'Connection': 'keep-alive', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'User-Agent': 'python-requests/2.5.0 CPython/2.7.8 Linux/3.18.5-201.fc21.x86_64'}


Server pak vrátil odpověď, která má také nějaké hlavičky...

In [4]:
print(odpoved.headers)

{'content-encoding': 'gzip', 'transfer-encoding': 'chunked', 'expires': 'Mon, 16 Feb 2015 10:00:50 GMT', 'server': 'GitHub.com', 'last-modified': 'Sun, 15 Feb 2015 19:38:20 GMT', 'cache-control': 'max-age=600', 'date': 'Mon, 16 Feb 2015 09:50:50 GMT', 'content-type': 'text/html; charset=utf-8'}


... a hlavně tělo odpovědi – v tomto případě HTML stránku:

In [5]:
print(odpoved.text[:200])  # text odpovědi

<!doctype html>
<html lang="cs">
<head>
    <meta charset="utf-8">

    <title>Python, programovací jazyk</title>
    <meta property="og:title" content="Python, programovací jazyk">

    <meta name="d


Když dáš v prohlížeči zobrazit zdrojový kód stránky http://python.cz, vypadat stejně.

Requests toho umí spoustu; kompletní dokumentace je na http://docs.python-requests.org.

# Twitter API

Normální internetové stránky jsou uzpůsobené pro lidské čtenáře. Existují ale i "stránky" dělané na to, aby je zpracovávaly programy. Vžilo se pro ně označení "API" (angl. Application Programming Interface, rozhraní pro programování aplikací). (Přesnější termín je "Webové API" nebo "REST API", protože "API" označovat jakékoli programátorské rozhraní – např. seznam metod pythoních slovníků je slovníkové API.)

Jedna ze stránek, které mají API, je Twitter. Jejich API je na stránce https://dev.twitter.com/rest/public. Zkusme ho použít: na stránce https://api.twitter.com/1.1/search/tweets.json by mělo jít hledat tweety:

In [6]:
odpoved = requests.get('https://api.twitter.com/1.1/search/tweets.json')
odpoved.text

'{"errors":[{"code":215,"message":"Bad Authentication data."}]}'

Aj, chyba! Twitter je zabezpečený. Musíme se přihlásit jako aplikace, což bohužel není vůbec jednoduché.

První krok je klasické přihlášení (nebo vytvoření účtu) na twitter.com. Potom jdi na stránku https://apps.twitter.com/, a vytvoř si tam aplikaci. (Jako jméno doporučuju třeba "*xyz*-test" kde *xyz* je tvoje jméno. Jako webovou adresu můžeš použít neexistující "http://test.example". Je taky potřeba doplnit dostatečně dlouhý popisek.)

Po vytvoření aplikace si otevři její záznam a jdi na záložku "Keys and Access Tokens". Tam najdeš dvě speciální hesla, kterým se můžeš přihlásit. Zkopíruj si je do Pythonu:

In [7]:
# Dosaď svoje údaje!
api_key = "D4HJp6PKmpon9eya1b2c3d4e5"
api_secret = "rhvasRMhvbuHJpu4MIuAb4WO50gnoQa1b2c3d4e5f6g7h8i9j0"

Tyto kódy je potřeba určitým způsobem spojit dohromady a poslat Twitteru, aby vytvořil *další* heslo, které pak půjde použít pro API. Ten způsob se naštěstí dá zapsat programem:

In [8]:
# Zakódování hesla
import base64
secret = '{}:{}'.format(api_key, api_secret)
secret64 = base64.b64encode(secret.encode('ascii')).decode('ascii')

# Vytvoření speciální hlavičky pro požadavek
headers = {
    'Authorization': 'Basic {}'.format(secret64),
    'Host': 'api.twitter.com',
}

# Odeslání požadavku.
# Předtím jsme použily funkci "requests.get", která stáhne informace ze serveru.
# Tady je metoda "requests.post", která serveru řekne aby provedl nějakou operaci.
# GET, POST, a ostatní HTTP metody jsou popsané na http://www.w3.org/Protocols/rfc2616/rfc2616-sec9.html
odpoved = requests.post('https://api.twitter.com/oauth2/token',
                        headers=headers,
                        data={'grant_type': 'client_credentials'})
odpoved.text

'{"token_type":"bearer","access_token":"AAAAAAAAAAAAAAAAAAAAAHhKXAAAAAAAaA1abB2bcC3cdD4deE5efF6fgG7ghH8hiI9ijJ0ja1b2c3d4e5f6g7h8i9j0a1b2c3d4e5f6g7h8i9j0"}'

Odpověď nám přišla jako řetězec, ve formátu JSON. JSON je ve světě vebových API častý, a tak má Requests přímo metodu, která JSON dekóduje. Můžeš si tak ušetřit psaní `import json` a `json.loads(odpoved.text)`.

Z odpovědi nás zajímá heslo jménem `access_token`.

In [9]:
bearer_token = odpoved.json()['access_token']
bearer_token

'AAAAAAAAAAAAAAAAAAAAAHhKXAAAAAAAaA1abB2bcC3cdD4deE5efF6fgG7ghH8hiI9ijJ0ja1b2c3d4e5f6g7h8i9j0a1b2c3d4e5f6g7h8i9j0'

Tohle heslo si můžeš zapsat přímo do programu, a jeho získávání příště přeskočit (místo všeho od `api_key = ...` napsat jen `berarer_token = 'AAAA...'`.

Blížíme se ke konci! Teď si uděláme si objekt `Session` s nastavenými přihlašovacími údaji. Tím řekneme knihovně Requests, aby tohle nové heslo používala pro další dotazy.

In [10]:
def bearer_auth(req):
    req.headers['Authorization'] = 'Bearer ' + bearer_token
    return req

session = requests.Session()
session.auth = bearer_auth

A teď když místo `requests.get` použiješ `session.get`, budeš přihlášená.

Zkus znovu stáhnout stránku https://api.twitter.com/1.1/search/tweets.json. Tentokrát s parametrem `'q'`, který říká co hledáš:

In [11]:
odpoved = session.get(
    'https://api.twitter.com/1.1/search/tweets.json',
    params={'q': '#vesmír'},
)

Zkus si vypsat `odpoved.json()`. Zjistíš, že je to slovník plný spooousty informací. Nejzajímavější z nich je pod klíčem `'statuses'`: *seznam* tweetů. Každý tweet je zase slovník spousty informací; samotný text je pod klíčem `'text'`.

Tweety se tedy dají vypsat takhle:

In [12]:
data = odpoved.json()

for tweet in data['statuses']:
    print(tweet['text'])

Zajímavá hypotéza o vlivu skryté #energie na trpasličí #galaxie. http://t.co/uiSRUA4ShF | #vesmír #temnaenergie
RT @RudyMarsCz: Probíhá na povrchu #Venuše aktivní sopečná činnost? Odpověď nabízí RudyMars http://t.co/8tKms9tdfv #vesmír #astronomie #Cze…
Probíhá na povrchu #Venuše aktivní sopečná činnost? Odpověď nabízí RudyMars http://t.co/8tKms9tdfv #vesmír #astronomie #Czech #sopky
RT @vjkombajn: Velký třesk možná nebyl...
http://t.co/Z22qVg8XmC #veda #vesmir
RT @vjkombajn: Velký třesk možná nebyl...
http://t.co/Z22qVg8XmC #veda #vesmir
RT @vjkombajn: Velký třesk možná nebyl...
http://t.co/Z22qVg8XmC #veda #vesmir
RT @vjkombajn: Velký třesk možná nebyl...
http://t.co/Z22qVg8XmC #veda #vesmir
RT @vjkombajn: Velký třesk možná nebyl...
http://t.co/Z22qVg8XmC #veda #vesmir
Velký třesk možná nebyl...
http://t.co/Z22qVg8XmC #veda #vesmir
Zoznámte sa: Toto sú ľudia, ktorí chcú opustiť #Zem a kolonizovať #Mars http://t.co/FlaSwMuQSC #vesmir #kolonizacia #planetaMars
RT @RudyMarsCz: Jak staré 

Na závěr varování: hesla z tohoto příkladu sice slouží jen pro čtení, ale pokud se dostanou do špatných rukou, dá se s nimi napáchat neplecha – Twitter ti pak může zablokovat účet, což může být docela nepříjemné.

Hesla tedy nikomu neukazuj. Pokud je přesto někdo nezvaný uvidí, jdi na https://apps.twitter.com a změň je (tlačítko "Regenerate Consumer Key and Secret" pod Keys and Access Tokens).